In [33]:
import pandas as pd
import sys
import os
import json
from bokeh.io import output_notebook, output_file, save
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.palettes import Category20
from bokeh.layouts import gridplot
from bokeh.models import HoverTool

In [2]:
# add the parent path so that utils can be imported
sys.path.insert(0, os.path.abspath('..'))

In [3]:
# setup auto reload for working on the util module
%load_ext autoreload
%autoreload 2

In [4]:
from vis.utils import read_rdf

In [5]:
output_notebook()

Loading BokehJS ...

In [16]:
dataframes = read_rdf('../metadata.json', char_types=['Aluminate Species'])

In [18]:
dataframes[0].head()

,r,RDF_Al-Ob,RDF_Al-Oh,RCN_Al-Ob,RCN_Al-Oh,dimer
0,0.00,0.0,0.0,0.0,0.0,(OH)3Al-O-Al(OH)32-+ 180 H2O + 2 Na+
1,0.02,0.0,0.0,0.0,0.0,(OH)3Al-O-Al(OH)32-+ 180 H2O + 2 Na+
2,0.04,0.0,0.0,0.0,0.0,(OH)3Al-O-Al(OH)32-+ 180 H2O + 2 Na+
3,0.06,0.0,0.0,0.0,0.0,(OH)3Al-O-Al(OH)32-+ 180 H2O + 2 Na+
4,0.08,0.0,0.0,0.0,0.0,(OH)3Al-O-Al(OH)32-+ 180 H2O + 2 Na+


In [30]:
# Create the base figure to plot to:
fig = figure()

# Create the hover tool
fig.add_tools(HoverTool(tooltips=[
    ("Radius", "@r"),
    ("Probability", "@{RDF_Al-Oh}"),
    ("Connectivity", "@{RCN_Al-Oh}")
]))

# Generate the colors to use, the number of dataframes is equal to the
# number of colored lines we want.
colors = Category20[len(dataframes)]

# Iterate through the list of dataframes
for (df, df_color) in zip(dataframes, colors):
    
    # Declare the source, this allows plots to be linked.
    fig_source = ColumnDataSource(df)
    
    # Declare the actual line plot.
    fig.line(
        source = fig_source,
        x='r',
        y='RDF_Al-Oh',
        legend='dimer',
        color=df_color,
        line_width=1.5
    )

# Display the figure.
show(fig)

## Linked RDF Plots

In [38]:
# Create the base figure to plot to:
left_fig = figure(title="RDF of Al-OH", plot_width=400)
right_fig = figure(title="RDF of Al-O (bending)", plot_width=400)

# Add the hover tool
left_fig.add_tools(HoverTool(tooltips=[
    ("Radius", "@r"),
    ("Probability", "@{RDF_Al-Oh}"),
    ("Connectivity", "@{RCN_Al-Oh}")
]))

right_fig.add_tools(HoverTool(tooltips=[
    ("Radius", "@r"),
    ("Probability", "@{RDF_Al-Ob}"),
    ("Connectivity", "@{RCN_Al-Ob}")
]))

# Generate the colors to use, the number of dataframes is equal to the
# number of colored lines we want.
colors = Category20[len(dataframes)]

# Iterate through the list of dataframes
for (df, df_color) in zip(dataframes, colors):
    
    # Declare the source, this allows plots to be linked.
    fig_source = ColumnDataSource(df)
    
    # Declare the actual line plot.
    left_fig.line(
        source = fig_source,
        x='r',
        y='RDF_Al-Oh',
        legend='dimer',
        color=df_color,
        line_width=1.5
    )
    
    # Declare the actual line plot.
    right_fig.line(
        source = fig_source,
        x='r',
        y='RDF_Al-Ob',
        legend='dimer',
        color=df_color,
        line_width=1.5
    )
    
plot = gridplot([[left_fig, right_fig]])

# Display the figure.
show(plot)